In [1]:
import pandas as pd

In [2]:
pixels_data = pd.read_csv('./fer2013/fer2013.csv')
pixels_data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [3]:
# 불필요한 데이터는 삭제합니다.
del pixels_data['Usage']
pixels_data.head()

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [4]:
emotion = pixels_data[['emotion']].copy()
emotion.head()

,emotion
0,0
1,0
2,2
3,4
4,6


In [5]:
del pixels_data['emotion']
pixels_data.head()

,pixels
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,151 150 147 155 148 133 111 140 170 174 182 15...
2,231 212 156 164 174 138 161 173 182 200 106 38...
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [6]:
type(pixels_data['pixels'][0])

str

In [7]:
pixels_data.dtypes

pixels    object
dtype: object

In [8]:
from sklearn import model_selection

In [9]:
train_data, test_data, train_label, test_label = model_selection.train_test_split(pixels_data, emotion, test_size=0.3, random_state=0)

In [10]:
test_label.head()

,emotion
34412,3
10642,4
13453,2
14843,4
30120,0


In [11]:
import numpy as np

In [12]:
train_data = np.array(list(map(str.split, train_data.pixels)), np.float32) 
test_data = np.array(list(map(str.split, test_data.pixels)), np.float32) 

In [13]:
train_data.max()

255.0

In [14]:
train_data = train_data / 255.0
test_data = test_data / 255.0

In [15]:
from sklearn.preprocessing import OneHotEncoder

In [16]:
enc = OneHotEncoder(categories='auto')

enc.fit(train_label)
train_label = enc.transform(train_label).toarray() # toarray까지 먹여야 완벽히 바뀐다.

enc.fit(test_label)
test_label = enc.transform(test_label).toarray()

print(train_label.shape)
print(test_label.shape)

(25120, 7)
(10767, 7)


In [17]:
import tensorflow as tf
from tensorflow import layers

import os
tf.logging.set_verbosity(tf.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [18]:
print(train_data.shape)

(25120, 2304)


In [19]:
X = tf.placeholder(tf.float32, [None, 2304]) # number of X_data's columns
Y = tf.placeholder(tf.float32, [None, 7]) # number of Y_data's columns (one-hot vector)

dropout_sign = tf.placeholder(tf.bool) # layers.dropout() 은 True/False 로 Training/Testing 여부를 결정해 줄 수 있습니다.

In [20]:
L1 = layers.dense(X, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal())
L1 = layers.dropout(L1, rate=0.2, training=dropout_sign) # layers.dropout()의 "rate"는 keeping rate가 아닌 dropping rate이며, default 값은 0.5입니다.

L2 = layers.dense(L1, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal()) 
L2 = layers.dropout(L2, rate=0.2, training=dropout_sign) 

L3 = layers.dense(L2, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal()) 
L3 = layers.dropout(L3, rate=0.2, training=dropout_sign) 

L4 = layers.dense(L3, 256, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.he_normal()) 
L4 = layers.dropout(L4, rate=0.2, training=dropout_sign) 

model = layers.dense(L4, 7, activation=None) # 2 == # of Y_data's columns

In [21]:
cost = tf.losses.softmax_cross_entropy(Y, model) 
optimizer = tf.train.AdamOptimizer(1e-2).minimize(cost) # 1e-2 == 0.01

In [22]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [23]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [24]:
total_epoch = 100
total_cost = 0 # cost

for epoch in range(total_epoch):
    train_cp = [] # Training accuracy 를 동시에 출력해보도록 합니다.
    
    # 매 Epoch마다 Total cost를 출력합니다.
    _, cost_val = sess.run([optimizer, cost], feed_dict={X: train_data, Y: train_label, dropout_sign: True}) # , dropout_sign: True
    total_cost += cost_val # cost

    # 매 Epoch마다 Training/Test accuracy를 출력합니다. (dropout_sign을 False로 바꾸어 dropout을 걷어내줘야 합니다.)
    train_cp += sess.run([is_correct], feed_dict={X: train_data, Y: train_label, dropout_sign: False}) # , dropout_sign: False
    test_accuracy = sess.run([accuracy], feed_dict={X: test_data, Y: test_label, dropout_sign: False})
    
    if epoch % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 
              '|| Avg. cost =', '{:.3f}'.format(total_cost / (epoch + 1)), # cost
              '|| Training accuracy : {:.3f}'.format(np.mean(train_cp)), # Training accuracy
              '|| Test accuracy : {:.3f}'.format(float(test_accuracy[0]))) # Test accuracy

print('Learning process is completed!')

Epoch: 0001 || Avg. cost = 2.594 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0011 || Avg. cost = 16.095 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0021 || Avg. cost = 9.309 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0031 || Avg. cost = 6.893 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0041 || Avg. cost = 5.653 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0051 || Avg. cost = 4.900 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0061 || Avg. cost = 4.394 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0071 || Avg. cost = 4.031 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0081 || Avg. cost = 3.757 || Training accuracy : 0.250 || Test accuracy : 0.252
Epoch: 0091 || Avg. cost = 3.543 || Training accuracy : 0.250 || Test accuracy : 0.252
Learning process is completed!
